In [1]:
import cv2, requests
import pandas as pd
import urllib.request
import numpy as np
import face_recognition
import os

In [15]:
df_person = pd.read_csv('./newdb/tagPerson.csv')
df_person.head(1)

personID                name
0         0  Chia-wei Woo (吳家瑋)

In [8]:
df_tag = pd.read_csv('./newdb/tag.csv')
result_df = df_tag[df_tag['personID'] == 0].dropna()
result_df.reset_index(inplace=True)


index  tagID  albumID photoID  personID     c1     c2    c3    c4
0    179    179    685.0   366-1       0.0  504.0  112.0  48.0  55.0

In [10]:
df_photo = pd.read_csv('./newdb/photo.csv')
df_photo['image'] = df_photo["imagepath"].map(lambda x: x.split('/')[2].split('.')[0])
result_df.head(1)

Empty DataFrame
Columns: [index, tagID, albumID, photoID, personID, c1, c2, c3, c4]
Index: []

In [5]:
photo_ID = []

for i in range(len(result_df)):
    id = result_df.loc[i, "photoID"]
    aid = result_df.loc[i, "albumID"]
    c1, c2, c3, c4 = (int(i) for i in result_df.iloc[i, -4:])
    temp = df_photo[(df_photo['image'] == id) & (df_photo['aID'] == aid)]
    if not temp.empty:
        url = "https://digitalimages.hkust.edu.hk/gallery/"+temp['imagepath'].values[0].replace(' ', '%20')
        with urllib.request.urlopen(url) as url:
            s = url.read()
        arr = np.asarray(bytearray(s), dtype=np.uint8)
        img = cv2.imdecode(arr, -1) # 'Load it as it is'
        c1 = int(c1 * (img.shape[1]/640))
        c2 = int(c2 * (img.shape[1]/640))
        c3 = int(c3 * (img.shape[1]/640))
        c4 = int(c4 * (img.shape[1]/640))

        cv2.rectangle(img, (c1, c2), (c1+c3, c2+c4), (0, 0, 255), 2) 

        face = img[c2:c2 + c4, c1:c1 + c3]

        cv2.imwrite('unknown/' + id + '_' + str(i+1) + '.jpg', face)
    photo_ID.append([id, ,id + '_' + str(i+1) + '.jpg', ])
id_list = np.array(photo_ID)

In [6]:
def read_img(path):
    img = cv2.imread(path)
    (h, w) = img.shape[:2]
    width = 500
    ratio = width / float(w)
    height = int(h * ratio)
    return cv2.resize(img, (width, height))


def face_rec():

  acc_list = []

  known_encodings = []
  known_names = []
  known_dir = 'known'

  for file in os.listdir(known_dir):
    img = read_img(known_dir + '/' + file)
    img_enc = face_recognition.face_encodings(img)[0]
    known_encodings.append(img_enc)
    known_names.append(file.split('.')[0])

  unknown_dir = 'unknown'
  for file in os.listdir(unknown_dir):
    print("Processing", file)
    img = read_img(unknown_dir + '/' + file)

      
    img_enc = face_recognition.face_encodings(img)
    if(img_enc == []):
      acc_list.append(0)
      continue
    results = face_recognition.compare_faces(known_encodings, img_enc[0])
    accuracy = np.sum(np.array(results)) / len(results)
    acc_list.append(accuracy)
  return np.array(acc_list)

def main():
  acc_list = face_rec()
  acc_list = acc_list >= 0.8
  return 




In [7]:
print(main())

Processing 1.jpg
Processing 10.jpg
Processing 11.jpg
Processing 14.jpg
Processing 15.jpg
Processing 16.jpg
Processing 17.jpg
Processing 18.jpg
Processing 19.jpg
Processing 2.jpg
Processing 20.jpg
Processing 21.jpg
Processing 22.jpg
Processing 23.jpg
Processing 24.jpg
Processing 25.jpg
Processing 26.jpg
Processing 27.jpg
Processing 28.jpg
Processing 29.jpg
Processing 3.jpg
Processing 30.jpg
Processing 31.jpg
Processing 32.jpg
Processing 33.jpg
Processing 34.jpg
Processing 35.jpg
Processing 36.jpg
Processing 37.jpg
Processing 38.jpg
Processing 39.jpg
Processing 4.jpg
Processing 40.jpg
Processing 41.jpg
Processing 42.jpg
Processing 43.jpg
Processing 44.jpg
Processing 5.jpg
Processing 6.jpg
Processing 7.jpg
Processing 8.jpg
Processing 9.jpg


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('bool'), dtype('<U19')) -> None